<a href="https://colab.research.google.com/github/carmenfontenla/AA_PRACTICA1_GRUPO_N-GRUPO/blob/main/Canciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Un generador de texto es un servicio que se utiliza para crear y desarrollar textos. En este caso, a partir de un archivo de texto que contiene las Top 100 canciones de Spotify del momento en español, vamos a crear la letra de una canción. Para ello hay que entrenar el programa para que detecte cuáles son las cosas en común que tienen todas las canciones para que tenga éxito.

In [1]:
import numpy as np
# original_distribution es una matriz 1D NumPy de valores de 
# probabilidad que deben sumar 1. La temperatura es un factor 
# que cuantifica la entropía de la distribución de salida.
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    # Devuelve una versión reponderada de la distribución original. 
    # Es posible que la suma de la distribución ya no sea 1, por lo 
    # que la dividiremos por su suma para obtener la nueva distribución
    return distribution / np.sum(distribution)

Genera una distribuccion de temperturas, es decir,  probabilidades de escoger ciertas palabras o en este caso las palabras mas comunes. si la temperatura es baja, las frases que se crearan seran muy predecibles y habra muy poca diferencia entre las distintas epochs de entrenamiento. Sin embargo,si las temperaturas son altas dan como resultado distribuciones de mayor entropia, que generan datos mas sorprendentes y no estructurados.

Antes de entrenar la red neuronal, vamos a realizar un preprocesamiento de la bbdd ya que todo esta contenido dentro de un archivo llamao SUPERNOVA SAIKO.txt

Vamos a crear un archivo donde cada estrofa (cada 4 versos) se considerara un archivo .txt de entrenamiento, las guardadremos dentro de una carperta llamada estrofas

In [2]:
import os 
os.mkdir("/content/estrofas/")

Oara comenzar guardadermos todos las estrofas en una lista , luego borraremos los nombres de las cancioness que estan escriytos en mayusculas y por ultimo borraremos \n que aaprece al final de cada verso.

In [4]:
with open ("/content/SUPERNOVA SAIKO.txt") as archivo:
  lineas=archivo.readlines()
print(lineas)
lineaspre=list()
for i in lineas:
  lineaspre.append(i[:-2])
print(lineaspre)
lineaspre2=list()
for i in lineaspre:
  if i.isupper():
    continue
  else:
    lineaspre2.append(i)
print(lineaspre2)

['SUPERNOVA SAIKO \n', 'Siempre me ha gustado como suenan las palabras\n', 'en el idioma del amor\n', 'Nunca me ha gustado como suenan las palabras\n', 'en el idioma del rencor\n', '\n', 'No me quiero ir de aquí sin probar\n', 'que se siente estar a flor de piel\n', 'no me quiero ir de aquí sin probar\n', 'la miel amarga del fracaso\n', '\n', 'Estoy esperando el momento para explotar\n', 'quiero ser una supernova\n', 'y brillar para siempre\n', '\n', 'Siempre he sentido que estoy fuera de lugar\n', 'en un mundo sin corazón\n', 'Nunca he sabido como encajar\n', 'en este mundo de ilusión\n', '\n', 'No me quiero ir de aquí sin probar\n', 'que se siente estar a flor de piel\n', 'no me quiero ir de aquí sin probar\n', 'la miel amarga del fracaso\n', '\n', 'Estoy esperando el momento para explotar\n', 'quiero ser una supernova\n', 'y brillar para siempre\n', '\n', '¿Por qué no puedo ser feliz con lo que tengo?\n', '¿Por qué siempre quiero más?\n', '¿Por qué no puedo vivir sin hacerme daño?\n

Ahora una vez que ya hemos completado este proceso, vamos a guardar dentro de archivos .txt cada estrofa. Para ello necesitamos calcular la longitud de las lineaspre2 y dividirlas entre 4 ya que queremos cada estrofa en uno distinto.

In [5]:
path="/content/estrofas/"
numestrofas=int(len(lineaspre2)/4)
print(numestrofas)

1488


Ahora vamos a correr el for para generar los archivos de texto.

In [6]:
for i in range(numestrofas):
  nombre=path+"letras"+str(i)+".txt"
  f=open(nombre,"a")
  f.write(lineaspre2[4*i]+" "+lineaspre2[4*i+1]+" "+lineaspre2[4*i+2]+" "+lineaspre2[4*i+3])


Creamos una carpeta llamada estrofas, y llamaremos  text_dataset_from_directory (conjunto de datos creados con la herramineta de keras) en el cual habbra todos los archivos de texto que indiquemos, en este caso estrofas.

Como siempre, obtenemos una carpeta llamada aclImdb que contiene dos subcarpetas, una para reseñas de películas con opiniones negativas y otra para reseñas con opiniones positivas. Hay un archivo de texto por revisión. Llamaremos a text_dataset_ from_directory con label_mode=None para crear un conjunto de datos que lea estos archivos y produzca el contenido de texto de cada archivo.

In [7]:
import tensorflow as tf
from tensorflow import keras
dataset = keras.utils.text_dataset_from_directory(
    directory="estrofas", label_mode=None, batch_size=256)


Found 1488 files belonging to 1 classes.


Antes de hacer la vectorizacion del texto, necesitamos una lista de cuantas palabras unicas hay, para saber el tamaño del vocaulario que tenemos.
Primero creamos una lista con todas las palabras y luego creamos otra lista con las palabras unicas, sin que haya repeticiones.

In [8]:
palabras=list()
for i in range(len(lineaspre2)):
  for j in lineaspre2[i].split():
    palabras.append(j)
print(palabras)

['Siempre', 'me', 'ha', 'gustado', 'como', 'suenan', 'las', 'palabra', 'en', 'el', 'idioma', 'del', 'amo', 'Nunca', 'me', 'ha', 'gustado', 'como', 'suenan', 'las', 'palabra', 'en', 'el', 'idioma', 'del', 'renco', 'No', 'me', 'quiero', 'ir', 'de', 'aquí', 'sin', 'proba', 'que', 'se', 'siente', 'estar', 'a', 'flor', 'de', 'pie', 'no', 'me', 'quiero', 'ir', 'de', 'aquí', 'sin', 'proba', 'la', 'miel', 'amarga', 'del', 'fracas', 'Estoy', 'esperando', 'el', 'momento', 'para', 'explota', 'quiero', 'ser', 'una', 'supernov', 'y', 'brillar', 'para', 'siempr', 'Siempre', 'he', 'sentido', 'que', 'estoy', 'fuera', 'de', 'luga', 'en', 'un', 'mundo', 'sin', 'corazó', 'Nunca', 'he', 'sabido', 'como', 'encaja', 'en', 'este', 'mundo', 'de', 'ilusió', 'No', 'me', 'quiero', 'ir', 'de', 'aquí', 'sin', 'proba', 'que', 'se', 'siente', 'estar', 'a', 'flor', 'de', 'pie', 'no', 'me', 'quiero', 'ir', 'de', 'aquí', 'sin', 'proba', 'la', 'miel', 'amarga', 'del', 'fracas', 'Estoy', 'esperando', 'el', 'momento', 'pa

In [9]:
unicas=list(set(palabras))
print(len(unicas))
print(len(palabras))


6621
38600


Como podemos ver de las 38600 palabras que tenemos en total, solo hay 6621 que no estan repetidas, es decir, palabras unicas, estas pasaran a formar parte del vocabulario.

Esta parte vectoriza el text, eso significa que tranforma una frase en un vector de numeros, cada entrada del vector simboliza una palabra, por eso es importante saber con cuantas palabras contamos.

In [ ]:
from tensorflow.keras.layers import TextVectorization
#Nuetsras secuencia mas larga tendra una longitud de 100 palabras
sequence_length = 100
#Decidimos trabajar co 2/3 del vocabulario total
vocab_size = 4414
text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    # Queremos devolver secuencias de índice 
    # de palabras con valores enteros
    output_mode="int",
    # Trabajaremos con entradas y objetivos de longitud 
    # 100 (pero como compensaremos los objetivos en 1, 
    # el modelo verá secuencias de longitud 99).
    output_sequence_length=sequence_length,
)
text_vectorization.adapt(dataset)

In [ ]:
def prepare_lm_dataset(text_batch):
    # Convertimos un lote de textos (cadenas) 
    # en un lote de secuencias enteras.
    vectorized_sequences = text_vectorization(text_batch)
    # Creamos entradas cortando la última palabra de las secuencias
    x = vectorized_sequences[:, :-1]
    # Creamos objetivos compensando las secuencias por 1.
    y = vectorized_sequences[:, 1:]
    return x, y

lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=4)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 2048
num_heads = 2
# De ejemplo
sequence_length = 100
vocab_size = 4414

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
# Softmax sobre posibles palabras de vocabulario, 
# calculado para cada timestep de secuencia de salida.
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

Se va a utilizar un callback para generar texto usando diferentes rangos de temperatura después de cada epoch, con el fin de evaluar cómo evoluciona el texto generado y el impacto de la temperatura en la estrategia de muestreo. El texto generado se iniciará con el indicador "El amor" para todas las generaciones.

In [ ]:
import numpy as np

# Dict (diccionario) que asigna o mapea índices de palabras 
# a cadenas, para usarlos para decodificar texto
tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

# Implementa el muestreo de variable-temperatura 
# a partir de una distribución de probabilidad
def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 # Indicación o prompt que usamos para 
                 # iniciar la generación de texto
                 prompt,
                 # Cuantas palabras generar
                 generate_length,
                 model_input_length,
                 # Rango de temperaturas 
                 # a usar para el muestreo
                 temperatures=(1.,),
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq
        vectorized_prompt = text_vectorization([prompt])[0].numpy()
        self.prompt_length = np.nonzero(vectorized_prompt == 0)[0][0]

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            # Al generar texto, comenzamos 
            # desde nuestro prompt o aviso.
            sentence = self.prompt
            for i in range(self.generate_length):
                # Alimentamos nuestro modelo con la secuencia actual
                tokenized_sentence = text_vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                # Recuperamos las predicciones del último timestep y las 
                # utilizamos para muestrear o probar una nueva palabra.
                next_token = sample_next(
                    # O [0, i, :]
                    predictions[0, self.prompt_length - 1 + i, :]
                )
                sampled_token = tokens_index[next_token]
                # Agregamos la nueva palabra a 
                # la secuencia actual y repetimos
                sentence += " " + sampled_token
            print(sentence)

prompt = "El amor"
text_gen_callback = TextGenerator(
    prompt,
    #Es importante que la longitud de la secuencia sea menor que sequence_length que se establecio en la vectorizacion
    generate_length=40,
    model_input_length=sequence_length,
    # Usaremos un rango diverso de 
    # temperaturas para muestrear texto, 
    # para demostrar el efecto de la 
    # temperatura en la generación de texto.
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

In [ ]:
model.fit(lm_dataset, epochs=200, callbacks=[text_gen_callback])

Epoch 1/200
6/6 [==============================] - ETA: 0s - loss: 7.5360 == Generating with temperature 0.2
El amor que granat la elige girl mi tu comer  y piel habla yeh pandora superao te si ese t cuánto oh habitació mi hoga   portó resistenci escucha busco la country manejando peri hizo heriste star estrell me tens
== Generating with temperature 0.5
El amor continú v sientas en cálid dij ya fosse dime chapéame casa remata lloraera yeah escuchó ella lejo contesta tiene tú vieran porque obviamente libreta ande comiéndote doli estas tatua patrás hagan yeahyeahyeahyeah mambo envueltos hipnotizante avis katy fama tercero ni
== Generating with temperature 0.7
El amor tigh [UNK] seria murieron rau enamorarte gritándom baby esa cab standl a malgastast you maestr perdid juego salí sabore escuchando yo lágrimas yeah cabr [UNK] hacer mi gaby sintiendo everything hace darte todo quitándote de rabia similar viaj volvieras movias
== Generating with temperature 1.0
El amor que partie esta high po

Como conclusion podemos observar 
1. Comparar los textos generados en distintas epochs (pprincipio medio y final), y con distintas temperaturas.
2. Las redes neuranles como complemento del proceso creativo, añadir el dilema moral que se ha generado por plagio.
